In [2]:
import os
import re
import json
import math
import time
import demoji
import curlify
import requests
import pandas as pd

import pickle
from scipy.special import softmax

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from tqdm.notebook import tqdm

all_stopwords = stopwords.words('english')
pd.set_option('display.max_colwidth', None)

bearer_token = "AAAAAAAAAAAAAAAAAAAAADSXjAEAAAAAQ32sCpTWcilVX%2BQj0BjOArOSYCE%3DUax6ZCizLRwwQcWQirExGQEDEB903dxBZKM4LAuGTMTvMa4jVP"

from datetime import date

today = date.today()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/crarojasca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
data = pd.read_csv("archive/The Climate Change Twitter Dataset.csv")

In [4]:
data.shape

(15789411, 10)

In [5]:
data.head()

,created_at,id,lng,lat,topic,sentiment,stance,gender,temperature_avg,aggressiveness
0,2006-06-06 16:06:42+00:00,6132,NaN,NaN,Weather Extremes,-0.097180,neutral,female,NaN,aggressive
1,2006-07-23 21:52:30+00:00,13275,-73.949582,40.650104,Weather Extremes,0.575777,neutral,undefined,-1.114768,aggressive
2,2006-08-29 01:52:30+00:00,23160,NaN,NaN,Weather Extremes,0.500479,neutral,male,NaN,aggressive
3,2006-11-07 02:46:52+00:00,57868,NaN,NaN,Weather Extremes,0.032816,neutral,male,NaN,aggressive
4,2006-11-27 14:27:43+00:00,304553,NaN,NaN,Importance of Human Intervantion,-0.090428,neutral,male,NaN,aggressive


In [6]:
data.topic.value_counts()

Global stance                                 4135619
Importance of Human Intervantion              2594941
Weather Extremes                              2464814
Politics                                      1809583
Undefined / One Word Hashtags                 1305118
Donald Trump versus Science                    996244
Seriousness of Gas Emissions                   903478
Ideological Positions on Global Warming        602695
Impact of Resource Overconsumption             496924
Significance of Pollution Awareness Events     479995
Name: topic, dtype: int64

In [8]:
data.groupby(["topic", "stance"]).id.count().to_frame()

id
topic                                      stance           
Donald Trump versus Science                believer   581485
                                           denier     232399
                                           neutral    182360
Global stance                              believer  3418290
                                           denier     140215
                                           neutral    577114
Ideological Positions on Global Warming    believer   358888
                                           denier      79466
                                           neutral    164341
Impact of Resource Overconsumption         believer   325151
                                           denier      32805
                                           neutral    138968
Importance of Human Intervantion           believer  2067505
                                           denier     119330
                                           neutral    408106
Politics                                   believer  1372888
                                           denier     113779
                                           neutral    322916
Seriousness of Gas Emissions               believer   573474
                                           denier     107749
                                           neutral    222255
Significance of Pollution Awareness Events believer   339123
                                           denier      11559
                                           neutral    129313
Undefined / One Word Hashtags              believer  1050281
                                           denier      36343
                                           neutral    218494
Weather Extremes                           believer  1205339
                                           denier     317741
                                           neutral    941734

In [9]:
s = requests.Session()
def getTweetId(id_):
    # Twitter Endpoint
    url = "https://api.twitter.com/2/tweets/{}".format(id_)
    
    
    # Auth Bearer no research access or premium for the moment
    headers = {
        "Authorization": "Bearer {}".format(bearer_token)
    }
    
    # Query parameters
    params = {
    #     'start_time': start_date,
    #     'end_time': end_date,
        'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
        'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
        'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
        'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
        'next_token': {}
    }
    
    try:
        time.sleep(6)
        req = requests.Request(
            "GET",
            url,
            params=params,
            headers=headers
        ).prepare()

        response = s.send(req)
        results = json.loads(response.text)
        return results
    except:
        print(json.loads(response.text))
        

In [20]:
res_tweets = []
for id_ in tqdm(data.id.unique().tolist()[:5]):
    res_tweets += [getTweetId(id_)]

  0%|          | 0/100 [00:00<?, ?it/s]

In [1]:
len(res_tweets)

NameError: name 'res_tweets' is not defined

In [19]:
res_tweets

[{'data': {'author_id': '53',
   'id': '6132',
   'public_metrics': {'retweet_count': 2,
    'reply_count': 7,
    'like_count': 11,
    'quote_count': 3},
   'text': "my dentist just explained to me that global warming isn't necessarily due to pollution.",
   'conversation_id': '6132',
   'created_at': '2006-06-06T16:06:42.000Z',
   'lang': 'en',
   'edit_history_tweet_ids': ['6132'],
   'reply_settings': 'everyone'},
  'includes': {'users': [{'created_at': '2006-04-01T01:55:49.000Z',
     'id': '53',
     'username': 'sara',
     'verified': False,
     'name': 'sara m. williams',
     'public_metrics': {'followers_count': 90090,
      'following_count': 690,
      'tweet_count': 2691,
      'listed_count': 1590},
     'description': ''}]}},
 {'errors': [{'value': '13275',
    'detail': 'Could not find tweet with id: [13275].',
    'title': 'Not Found Error',
    'resource_type': 'tweet',
    'parameter': 'id',
    'resource_id': '13275',
    'type': 'https://api.twitter.com/2/proble

In [12]:
pd.DataFrame([getTweetId(6132), getTweetId(13275)])

,data,includes,errors
0,"{'created_at': '2006-06-06T16:06:42.000Z', 'author_id': '53', 'public_metrics': {'retweet_count': 2, 'reply_count': 7, 'like_count': 11, 'quote_count': 3}, 'edit_history_tweet_ids': ['6132'], 'id': '6132', 'text': 'my dentist just explained to me that global warming isn't necessarily due to pollution.', 'source': 'Twitter SMS', 'conversation_id': '6132', 'lang': 'en', 'reply_settings': 'everyone'}","{'users': [{'public_metrics': {'followers_count': 90090, 'following_count': 690, 'tweet_count': 2691, 'listed_count': 1590}, 'verified': False, 'username': 'sara', 'name': 'sara m. williams', 'description': '', 'created_at': '2006-04-01T01:55:49.000Z', 'id': '53'}]}",NaN
1,NaN,NaN,"[{'value': '13275', 'detail': 'Could not find tweet with id: [13275].', 'title': 'Not Found Error', 'resource_type': 'tweet', 'parameter': 'id', 'resource_id': '13275', 'type': 'https://api.twitter.com/2/problems/resource-not-found'}]"


In [ ]:
tweet_res["data"]

In [11]:
tweet_res

{'data': {'public_metrics': {'retweet_count': 2,
   'reply_count': 7,
   'like_count': 11,
   'quote_count': 3},
  'author_id': '53',
  'lang': 'en',
  'edit_history_tweet_ids': ['6132'],
  'conversation_id': '6132',
  'source': 'Twitter SMS',
  'reply_settings': 'everyone',
  'text': "my dentist just explained to me that global warming isn't necessarily due to pollution.",
  'created_at': '2006-06-06T16:06:42.000Z',
  'id': '6132'},
 'includes': {'users': [{'name': 'sara m. williams',
    'public_metrics': {'followers_count': 90090,
     'following_count': 690,
     'tweet_count': 2691,
     'listed_count': 1590},
    'description': '',
    'created_at': '2006-04-01T01:55:49.000Z',
    'username': 'sara',
    'id': '53',
    'verified': False}]}}

In [23]:
import pandas as pd

data = pd.read_csv("Twitter_scrapped/climateScam/@ClimateScam_CARDS_predictions.csv")

In [25]:
data.created_at.min()

'2022-11-17T13:48:42.000Z'